# 2016 MC cross-section corrections

In [12]:
%cd '/home/naodell/work/CMS/amumu'
%matplotlib notebook

from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nllfitter.plot_tools as pt

/home/naodell/work/CMS/amumu


Carry out some initial configuration.

In [13]:
ntuple_dir  = 'data/flatuples/mumu_2016'
lumi        = 36.4e3
selection   = ('mumu', 'combined')
period      = 2016
output_path = 'plots/fits/{0}_{1}'.format('_'.join(selection), period)

The predominant samples are ttbar and Drell-Yan.  The Drell-Yan sample should be split into $10 < M < 50$ and $M > 50$.  If it makes sense, I will also apply corrections based on jet multiplicity.

In [14]:
datasets    = [
               'ttbar_lep',                      
               'zjets_m-50',  'zjets_m-10to50',  
               'z1jets_m-50', 'z1jets_m-10to50', 
               'z2jets_m-50', 'z2jets_m-10to50', 
               'z3jets_m-50', 'z3jets_m-10to50', 
               'z4jets_m-50', 'z4jets_m-10to50', 
              ]

The template fit will be carried out over several 1D histograms which are specified here.

In [15]:
features = [
            'lepton1_pt', 'lepton1_eta', 'lepton1_phi',
            'lepton2_pt', 'lepton2_eta', 'lepton2_phi',
            'lepton_delta_eta', 'lepton_delta_phi', 'lepton_delta_r',
            'dilepton_mass', 'dilepton_pt', 'dilepton_eta', 'dilepton_phi',

            'met_mag',
            'n_jets', 'n_fwdjets', 'n_bjets', 
            #'bjet_pt', 'bjet_eta', 'bjet_phi', 'bjet_d0',
            #'jet_pt', 'jet_eta', 'jet_phi', 'jet_d0', 
            #'dijet_mass', 'dijet_pt', 'dijet_eta', 'dijet_phi', 
            #'dijet_pt_over_m',

            #'lepton1_b_mass', 'lepton1_b_pt', 
            #'lepton1_b_delta_eta', 'lepton1_b_delta_phi', 'lepton1_b_delta_r',
            #'lepton2_b_mass', 'lepton2_b_pt', 
            #'lepton2_b_delta_eta', 'lepton2_b_delta_phi', 'lepton2_b_delta_r',

            #'dilepton_j_mass', 'dilepton_j_pt', 
            #'dilepton_j_delta_eta', 'dilepton_j_delta_phi', 'dilepton_j_delta_r',
            #'dilepton_b_mass', 'dilepton_b_pt', 
            #'dilepton_b_delta_eta', 'dilepton_b_delta_phi', 'dilepton_b_delta_r',
            #'four_body_mass',
            #'four_body_delta_phi', 'four_body_delta_eta', 'four_body_delta_r',

            #'t_xj', 't_xb', 't_bj'
           ]

It's important to distinguish between continuously valued and nominal variables since they will need to be preprocessed differently.  A few preselection cuts will be required,

In [16]:
cuts     = 'lepton1_pt > 25 and abs(lepton1_eta) < 2.1 \
            and lepton2_pt > 25 and abs(lepton2_eta) < 2.1 \
            and lepton1_q != lepton2_q and 12 < dilepton_mass < 70'

Now we'll load the data that is saved in flat ntuples using a DataManager class,

In [17]:
data_manager = pt.DataManager(input_dir     = ntuple_dir,
                              dataset_names = datasets,
                              selection     = selection[0],
                              period        = period,
                              scale         = lumi,
                              cuts          = cuts
                             )

Loading dataframes: 100%|███████████████| 11.0/11.0 [00:02<00:00, 3.55it/s]
